## Demonstrate interactions with relations using omnikeeper

In [ ]:
import os
import omnikeeper_client as okc
from omnikeeper_client import TraitDefinition, TraitAttributeDefinition, TraitRelationDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER
import pandas as pd

As usual create the omnikeeper client

In [ ]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

In this example we will work with two traits host and interface and we will define relations using okc public functions. 

First lets define the trait names and layer.

In [ ]:
host_trait_name = "python_client_demo.host"
interface_trait_name = "python_client_demo.interface"
layer_name = "relations"

Make sure that traits are created correctly

In [ ]:
is_host_trait_created = okc.upsert_trait(okapiclient, TraitDefinition(host_trait_name, [
        TraitAttributeDefinition("id", "host.id", ATTRIBUTETYPE_INTEGER),
      ], optional_relations = [
        TraitRelationDefinition(identifier = "has_interface", predicate_id = "has_interface", direction_forward=True)
      ]))
print(is_host_trait_created)

is_interface_trait_created = okc.upsert_trait(okapiclient, TraitDefinition(interface_trait_name, [
        TraitAttributeDefinition("id", "interface.id", ATTRIBUTETYPE_INTEGER),
      ], optional_relations = [
        TraitRelationDefinition(identifier = "attached_to_host", predicate_id = "has_interface", direction_forward=False)
      ]))
print(is_interface_trait_created)

Create the host and interface CIs

Create host traitentities

In [ ]:
hosts_list = [
    {"id": 1},
    {"id": 2},
    {"id": 3},
]
okc.bulk_replace_trait_entities_by_filter_list(okapiclient, trait_name=host_trait_name, input=hosts_list, id_attributes=["id"], id_relations=[], write_layer=layer_name, filter={})

Create Interface traitentities

In [ ]:
interfaces_list = [
    {"id": 1},
    {"id": 2}, 
    {"id": 3}
]
okc.bulk_replace_trait_entities_by_filter_list(okapiclient, trait_name=interface_trait_name, input=interfaces_list, id_attributes=["id"], id_relations=[], write_layer=layer_name, filter={})

Fetch the created trait entities

In [ ]:
hosts = okc.get_all_traitentities_list(okapiclient, trait_name=host_trait_name, layers=[layer_name])
hosts

In [ ]:
interfaces = okc.get_all_traitentities_list(okapiclient, trait_name=interface_trait_name, layers=[layer_name])
interfaces

Add host and interface relations using the created cis and okc

In [ ]:
okc.add_trait_relations_by_ciid(okapiclient, host_trait_name, "has_interface", hosts[0]["ciid"], [interfaces[0]["ciid"], interfaces[1]["ciid"]], layer_name)
okc.add_trait_relations_by_ciid(okapiclient, host_trait_name, "has_interface", hosts[1]["ciid"], [interfaces[2]["ciid"]], layer_name)

### Relations operations using lists

Fetch has_host realtions. In the result interfaces will be as base_cis and hosts as related cis

In [ ]:
attached_to_host_relations = okc.get_trait_relation_list(okapiclient, trait_name=interface_trait_name, relation_name="attached_to_host", layers=[layer_name])
attached_to_host_relations

Fetch has_interface relations for all hosts

In [ ]:
has_interface_relations = okc.get_trait_relation_list(okapiclient, trait_name=host_trait_name, relation_name="has_interface", layers=[layer_name])
has_interface_relations

Merge hosts with realted cis in this case interfaces

In [ ]:
interface_dict = {interface['ciid']: interface for interface in interfaces}

hosts_with_relations = []
for host in hosts:
    merged_dict = host.copy()
    merged_dict['related_ciis'] = []
    for relation in has_interface_relations:
        if host['ciid'] == relation['base_ciid']:
            for related_ciid in relation['related_ciids']:
                if related_ciid in interface_dict:
                    merged_dict['related_ciis'].append(interface_dict[related_ciid])
    hosts_with_relations.append(merged_dict)

hosts_with_relations

The same can be done for interfaces to fetch its relations

In [ ]:
host_dict = {host['ciid']: host for host in hosts}

interfaces_with_relations = []
for interface in interfaces:
    merged_dict = interface.copy()
    merged_dict['related_ciis'] = []
    for relation in attached_to_host_relations:
        if interface['ciid'] == relation['base_ciid']:
            for related_ciid in relation['related_ciids']:
                if related_ciid in host_dict:
                    merged_dict['related_ciis'].append(host_dict[related_ciid])
    interfaces_with_relations.append(merged_dict)

interfaces_with_relations

### Relations operations using pandas dataframes

We will do the same steps here but using pandas dataframes to work with omnikeeper data

In [ ]:
has_interface_relations_df = okc.get_trait_relation_dataframe(okapiclient, trait_name=host_trait_name, relation_name="has_interface", layers=[layer_name])
has_interface_relations_df

Fetch hosts and interfaces using dataframes 

In [ ]:
hosts_df = okc.get_all_traitentities_dataframe(okapiclient, trait_name=host_trait_name, layers=[layer_name])
interfaces_df = okc.get_all_traitentities_dataframe(okapiclient, trait_name=interface_trait_name, layers=[layer_name])
hosts_df

In [ ]:
hosts_df.index.name

In [ ]:
has_interface_relations_df.index.name

Now merge the related data. In this case we can see how easy it is to merge related data using pandas dataframes

In [ ]:
hosts_with_relations_df = hosts_df.merge(has_interface_relations_df, left_index=True, right_index=True, how='outer')
hosts_with_relations_df

We can create a multi index dataframe which includes interfaces information

In [ ]:
df_exploded = hosts_with_relations_df.explode('related_ciids').reset_index()
df_exploded = df_exploded.rename(columns={df_exploded.columns[0]: 'ciid'})
df_exploded

In [ ]:
merged_df = pd.merge(df_exploded, interfaces_df, left_on='related_ciids', right_on='ciid', how='left', suffixes=('_host', '_interface'))
merged_df.set_index(['ciid', 'related_ciids'], inplace=True)
merged_df